In [23]:
import json
import requests
import pandas as pd
# json.load(urllib.request("url"))

In [29]:
def request_data(i):

    r = requests.get('https://medium.com/_/api/home-feed?limit=10&to='+str(i))
    return json.loads(r.text[16:])

In [30]:
medium_jsondata = []

for i in range(0,20):
    medium_jsondata.append(request_data(i))

In [179]:
medium_user = []
medium_blog = []
for medium_data in medium_jsondata:
#     print(medium_data)
    medium_dataUser = medium_data["payload"]["references"]["User"]
    for users in medium_dataUser:
        user = medium_dataUser[users]
        medium_user.append({
            "name":user["name"],
            "userId":user["userId"],
            "createdAt":user["createdAt"],
            "bio":user["bio"],
            "mediumMemberAt":user["mediumMemberAt"],
            "type":user["type"]
        })
        
    medium_dataBlog = medium_data["payload"]["references"]["Post"]
    for blogs in medium_dataBlog:
        blog = medium_dataBlog[blogs]
        medium_blog.append({
            "title":blog["title"],
            "createdAt":blog["createdAt"],
            "creatorId":blog["creatorId"],
            "id":blog["id"],
            "metaDescription":blog["virtuals"]["metaDescription"],
            "subtitle":blog["virtuals"]["subtitle"],
            "tags":' '.join([tags["slug"] for tags in blog["virtuals"]["tags"]])
        })

# drop_duplicates

In [41]:
# ## User 
# name
# userId
# createdAt
# bio

# ## Post
# title
# createdAt
# creatorId
# id # bog id
# virtuals(object)
#     metaDescription
#     subtitle
#     tags(array)
#         slug
        


In [180]:
blogs = pd.DataFrame(medium_blog).drop_duplicates(subset=["creatorId", "id"], keep="first")
users = pd.DataFrame(medium_user).drop_duplicates(subset=["userId"], keep="first")



In [61]:
blogs["creatorId"].unique() in users["userId"].unique()

True

In [183]:
blogs.head()

,createdAt,creatorId,id,metaDescription,subtitle,tags,title
0,1544254461478,a8634106cb64,2524c43bb922,Giving blood in the U.S. today is like joining...,"On paper, I know why I’m not eligible to give ...",health lgbtq equality gay memoir,What it Means to Donate Blood as a Gay Man in ...
1,1548508970826,64dc0564597a,e11c38f98977,"In Venezuela, anger over economic inequality f...",The institutions that should have protected th...,venezuela government politics economics donald...,What the Crisis in Venezuela Tells Us About De...
2,1539528932652,2879ca55026a,6052b0cc43ac,Some people thrive on taking advantage of ever...,Efficiency is a matter of perspective,money simplicity complexity machine-learning,How to Fight Against Complexity Bias
3,1533734807493,73720195c226,daaaaff085c,Ten years after I served in Operation Iraqi Fr...,"Vonnegut had Dresden, and I had Fallujah",veterans war life iraq kurt-vonnegut,Making Sense of My Time at War
4,1549487704191,78e429aad85a,f52733f4666,,"It has the cash, and Netflix has the audience....",netflix apple predictions business analysis,8 Reasons Why Apple Won’t Buy Netflix


In [63]:
users.head()

,bio,createdAt,mediumMemberAt,name,type,userId
0,"Freelance writer, journalist, and author of Ge...",1537835275432,0,Kristin Wong,User,b5f15686d60
1,"Technophile, technophobe. Former VP of Communi...",1494900815468,1533103311000,Jessica Powell,User,7bcec4b8c02e
2,Working on a memoir about gay pride and machis...,1540353674610,0,Edgar Gomez,User,a8634106cb64
3,Mike is an Assistant Professor of Management f...,1531584419456,1531551600000,"Michael Greiner, PhD, JD",User,64dc0564597a
4,"I am a Principal Scientist at Google, working ...",1537720406833,1537888274468,Vincent Vanhoucke,User,2879ca55026a


In [152]:
## blog visited rating movie set
visited = pd.read_csv("../data/raw/ratings.csv")

# userData = (visited["userId"]/17).map(int)

# blogData = (visited["movieId"]/4599).map(int)

# visited["user"] = userData
# visited["blog"] = blogData

In [134]:
visited.drop(["userId", "movieId", "timestamp"], axis=1, inplace=True)

In [117]:
# users["userId"]

In [162]:
visited['userId'] = visited["userId"].apply(lambda x: users["userId"][x])
visited['movieId'] = visited["movieId"].apply(lambda x: blogs["id"][x])




In [123]:
visited.to_csv("../data/processed/visited.csv", index=False)
blogs.to_csv("../data/processed/blogs.csv", index=False)
users.to_csv("../data/processed/users.csv", index=False)

In [145]:
(visited["movieId"]/300).map(int).value_counts()

1      413
0      371
4      207
3      183
6      161
2      149
5      127
8      122
7      121
9      111
11      93
10      89
12      83
13      41
19      38
16      32
29      30
14      28
18      25
21      20
23      19
24      17
409     14
28      13
22      12
15      11
17      11
305     10
92       9
27       9
      ... 
464      1
428      1
183      1
396      1
390      1
366      1
356      1
350      1
340      1
328      1
536      1
538      1
542      1
560      1
175      1
165      1
163      1
157      1
137      1
127      1
107      1
25       1
591      1
517      1
527      1
541      1
495      1
600      1
570      1
617      1
Name: movieId, Length: 243, dtype: int64

In [161]:
(visited["movieId"]/300).map(int).value_counts()
visited = visited[(visited["movieId"] < 43) & (visited["userId"] < 36)][["userId", "movieId", "rating"]]

In [154]:
users.shape, blogs.shape

((36, 6), (43, 7))

In [178]:
# visited.rename(columns={"blogId":"movieId"}, inplace=True)
visited.to_csv("../data/processed/rating.csv", index=False)

In [173]:
visited

,userId,blogId,rating
0,7bcec4b8c02e,e11c38f98977,4.0
1,7bcec4b8c02e,daaaaff085c,4.0
2,7bcec4b8c02e,66771889da10,4.0
261,64dc0564597a,b4295a07afdc,0.5
300,2879ca55026a,1a0a0f6e69cb,3.0
301,2879ca55026a,eacf251d78ad,2.0
516,73720195c226,e11c38f98977,4.0
517,73720195c226,1a0a0f6e69cb,4.0
518,73720195c226,40347c6ed448,4.0
519,73720195c226,7588781774b3,4.0


In [185]:
blogs

,createdAt,creatorId,id,metaDescription,subtitle,tags,title
0,1544254461478,a8634106cb64,2524c43bb922,Giving blood in the U.S. today is like joining...,"On paper, I know why I’m not eligible to give ...",health lgbtq equality gay memoir,What it Means to Donate Blood as a Gay Man in ...
1,1548508970826,64dc0564597a,e11c38f98977,"In Venezuela, anger over economic inequality f...",The institutions that should have protected th...,venezuela government politics economics donald...,What the Crisis in Venezuela Tells Us About De...
2,1539528932652,2879ca55026a,6052b0cc43ac,Some people thrive on taking advantage of ever...,Efficiency is a matter of perspective,money simplicity complexity machine-learning,How to Fight Against Complexity Bias
3,1533734807493,73720195c226,daaaaff085c,Ten years after I served in Operation Iraqi Fr...,"Vonnegut had Dresden, and I had Fallujah",veterans war life iraq kurt-vonnegut,Making Sense of My Time at War
4,1549487704191,78e429aad85a,f52733f4666,,"It has the cash, and Netflix has the audience....",netflix apple predictions business analysis,8 Reasons Why Apple Won’t Buy Netflix
5,1549474238519,9cca1cc5944f,bc30074fb93d,,Here’s how the new smartphone tech works,tech huawei smartphones cameras android,Your Next Phone May Have a Hole in the Screen
6,1548927145568,96d8c816c5db,66771889da10,"Customers are willing to pay upwards of $1,000...","Customers are willing to pay upwards of $1,000...",apple business psychology economics product-ma...,The Simple Psychology Behind Apple’s Fall
7,1548889893092,486aefe3d462,ebbcb4723c0e,A chance meeting between a man with idle machi...,The untold origin story of an iconic workout m...,fitness oprah exercise history marketing,From Oil to Oprah: An Oral History of the Stai...
8,1549415160330,b42354b051f1,b5b8c7911819,,A small group of well-educated professionals e...,technology economics work inequality,Tech Is Splitting the U.S. Workforce in Two
9,1549314322301,7da2a328fd85,ecfca5ab7518,,Great news! You can be a creator. Follow these...,creativity productivity self how-to,How to Be Creative


In [184]:
blogs.to_csv("../data/processed/blogs.csv", index=False)